In [1]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.matlib import repmat
import PyQt5
import sys

ctfile1 = "ciphertext-00112233445566778899aabbccddeeff.txt"
ctfile2 = "ciphertext-unknown_key.txt"
ptfile1 = "plaintext-00112233445566778899aabbccddeeff.txt"
ptfile2 = "plaintext-unknown_key.txt"
trfile1 = "traces-00112233445566778899aabbccddeeff.bin"
trfile1 = "traces-00112233445566778899aabbccddeeff.npy"
trfile2 = "traces-unknown_key.bin"
trfile2 = "traces-unknown_key.npy"

Sbox = np.array([
            0x63, 0x7C, 0x77, 0x7B, 0xF2, 0x6B, 0x6F, 0xC5, 0x30, 0x01, 0x67, 0x2B, 0xFE, 0xD7, 0xAB, 0x76,
            0xCA, 0x82, 0xC9, 0x7D, 0xFA, 0x59, 0x47, 0xF0, 0xAD, 0xD4, 0xA2, 0xAF, 0x9C, 0xA4, 0x72, 0xC0,
            0xB7, 0xFD, 0x93, 0x26, 0x36, 0x3F, 0xF7, 0xCC, 0x34, 0xA5, 0xE5, 0xF1, 0x71, 0xD8, 0x31, 0x15,
            0x04, 0xC7, 0x23, 0xC3, 0x18, 0x96, 0x05, 0x9A, 0x07, 0x12, 0x80, 0xE2, 0xEB, 0x27, 0xB2, 0x75,
            0x09, 0x83, 0x2C, 0x1A, 0x1B, 0x6E, 0x5A, 0xA0, 0x52, 0x3B, 0xD6, 0xB3, 0x29, 0xE3, 0x2F, 0x84,
            0x53, 0xD1, 0x00, 0xED, 0x20, 0xFC, 0xB1, 0x5B, 0x6A, 0xCB, 0xBE, 0x39, 0x4A, 0x4C, 0x58, 0xCF,
            0xD0, 0xEF, 0xAA, 0xFB, 0x43, 0x4D, 0x33, 0x85, 0x45, 0xF9, 0x02, 0x7F, 0x50, 0x3C, 0x9F, 0xA8,
            0x51, 0xA3, 0x40, 0x8F, 0x92, 0x9D, 0x38, 0xF5, 0xBC, 0xB6, 0xDA, 0x21, 0x10, 0xFF, 0xF3, 0xD2,
            0xCD, 0x0C, 0x13, 0xEC, 0x5F, 0x97, 0x44, 0x17, 0xC4, 0xA7, 0x7E, 0x3D, 0x64, 0x5D, 0x19, 0x73,
            0x60, 0x81, 0x4F, 0xDC, 0x22, 0x2A, 0x90, 0x88, 0x46, 0xEE, 0xB8, 0x14, 0xDE, 0x5E, 0x0B, 0xDB,
            0xE0, 0x32, 0x3A, 0x0A, 0x49, 0x06, 0x24, 0x5C, 0xC2, 0xD3, 0xAC, 0x62, 0x91, 0x95, 0xE4, 0x79,
            0xE7, 0xC8, 0x37, 0x6D, 0x8D, 0xD5, 0x4E, 0xA9, 0x6C, 0x56, 0xF4, 0xEA, 0x65, 0x7A, 0xAE, 0x08,
            0xBA, 0x78, 0x25, 0x2E, 0x1C, 0xA6, 0xB4, 0xC6, 0xE8, 0xDD, 0x74, 0x1F, 0x4B, 0xBD, 0x8B, 0x8A,
            0x70, 0x3E, 0xB5, 0x66, 0x48, 0x03, 0xF6, 0x0E, 0x61, 0x35, 0x57, 0xB9, 0x86, 0xC1, 0x1D, 0x9E,
            0xE1, 0xF8, 0x98, 0x11, 0x69, 0xD9, 0x8E, 0x94, 0x9B, 0x1E, 0x87, 0xE9, 0xCE, 0x55, 0x28, 0xDF,
            0x8C, 0xA1, 0x89, 0x0D, 0xBF, 0xE6, 0x42, 0x68, 0x41, 0x99, 0x2D, 0x0F, 0xB0, 0x54, 0xBB, 0x16
            ])

In [2]:
#################################################
#                                               #
#                                               #
#        C O D E   S T A R T S   H E R E        #
#                                               #
#                                               #
#################################################

tohex = np.vectorize(lambda x: int(x, 16))

def myin(filename):
    with open(filename, 'r') as f:
        data = f.readlines()[:-1]
    return tohex(np.array([line[:-2].split(" ") for line in data]))    

def myload(filename):
    return np.load(filename)

def myload2(fname,trlen=370000,start=0,len=370000,n=200):
    myfile = open(fname, 'rb')
    traces = np.zeros((n, len))
    myfile.seek(start)
    for i in range(n):      
        if len+start > trlen:
            t = [int.from_bytes(myfile.read(1), byteorder='big') for i in range(len-start)]
        else:
            t = [int.from_bytes(myfile.read(1), byteorder='big') for i in range(len)]
        traces[i] = t
    myfile.close()
    return traces

def mycorr(x,y):
    xr,xc = x.shape
    yr,yc = y.shape
    assert xr==yr, "Matrix row count mismatch"     

    x = x - x.mean(0)
    y = y - y.mean(0)
    C = x.T.dot(y)
    xsq = np.atleast_2d(np.sqrt(np.sum(x**2, 0)))
    ysq = np.atleast_2d(np.sqrt(np.sum(y**2, 0)))
    C = np.divide(C, repmat(xsq.T, 1, yc))
    C = np.divide(C, repmat(ysq, xc, 1))
    return C


In [3]:
#TODO:
#Select which files to open. Filenames defined at the top.
#Functions for loading are named the same as in the Matlab code.
traces = myload(trfile2)
plaintexts = myin(ptfile2)
ciphertexts = myin(ctfile2)

print(plaintexts[0])

#Use this to check that you loaded the files correctly
print("traces", traces.shape)
print("plaintexts", plaintexts.shape)

#TODO:
#After doing the next part you can come back here and change the
#start and stop values to remove the parts of the trace we don't need.
#E.g. let's make up some numbers. If the leakage window starts at x = 123456
#and the window ends at x = 246912 you set start and stop to those values
#respectively
# start = 45000
# stop = 77000 #traces.shape[1]
start = 0
stop = 40000
traces = traces[:, start:stop]

#TODO:
#plot one of the power traces.
#Try to determine if you can see the 10 rounds of AES
#in the traces. Also, try to determine if keybytes are
#calculated in series (8-bit operations), 4 at a time
#(32-bit operations), or all parallely.
# plt.plot(traces)
%matplotlib qt
plt.plot(np.arange(start, stop), traces[1])
plt.show()

#TODO:
#Plot only the first round (or two) of AES.
#Also plot vertical lines around where you think we will
#find the information leakage. To plot vertical lines
#you can use the command
#plt.axvline(x, color, xmin, xmax, linestyle)
#
#You may wish to delimit your traces around where
#the leakage point is to speed up computations later

#Tip: to get resizeable popup plots in jupyter notebook
#you can tell the backend to use a library such as 
#qt to handle the plotting in a new window. 
#Test this following code to see how:
#
#%matplotlib qt 
#plt.plot([1,2,3])

[ 41  35 190 132 225 108 214 174  82 144  73 241 241 187 233 235]
traces (149, 550000)
plaintexts (149, 16)


In [5]:
# plt.clf()

#TODO:
#for each possible value of the keybytes, formulate
#a power hypothesis. Make sure you understand what
#the power hypothesis represents and why we need it
#
#The Sbox for AES is provided in case you wish to 
#use it for your power hypothesis

number_of_traces = traces.shape[0]
number_of_measurements = traces.shape[1]


#TODO:
#for each of the 16 subkeys, use your power hypothesis
#to calculate CC using the mycorr function.
#This function is an exact copy of the matlab function.
#
#The instructions say to make sure you know what CC is.
#Obviously one of the Cs is correlation. Which of these
#describes what CC is:
#1) Cross Correlation
#2) Correlation Coefficients
#3) Cumulative Correlation
#4) Central Correlation
#
#What shape would you expect the CC variable to have?
#Try plotting its shape and see if your guess was correct.

key_arr = np.zeros(16)
corrs = np.zeros(16)
for BYTE in range(0, 16):
    powerhyp = np.zeros(shape=(number_of_traces, 256))
    for idx, (trace, plaintext) in enumerate(zip(traces, plaintexts)):
        plaintext_byte = plaintext[BYTE]
        for key_byte in range(256):
            intermediate_value = Sbox[key_byte ^ plaintext_byte]
            hw = sum([int(bit) for bit in bin(intermediate_value)[2:]])
            powerhyp[idx, key_byte] = hw
    #YOUR CODE HERE#

    # powerhyp
    # powerhyp[index, key_byte] = the estimated power consumption of the encryption of plaintext #index assuming that we have key byte key_byte

    # print(powerhyp[1:10, 1:10])
    CC = mycorr(powerhyp, traces)
    # print(CC.shape)

    # CC[x, y] = correlation between power consumption and power hypothesis at the point in time y and the guessed key byte x

    # #TODO:
    # #Write code to find the correct keybyte.
    # #You will want to print it. Depending on your code
    # #you may have to print it inside the loop.

    # for key_byte in range(256):
    #     plt.plot(CC[key_byte, :]) 
    # plt.plot(CC[0x11, :]) # correlation if correct key byte is 01
    # plt.plot(CC[2])
    # plt.plot(CC[0x11])
    # plt.show()

    max_corr_idx = np.argmax(np.abs(CC))
    # print(max_corr_idx)
    key_byte = max_corr_idx // number_of_measurements
    key_arr[BYTE] = key_byte
    corrs[BYTE] = np.max(np.abs(CC))
    print("%02x" % key_byte, end='')
    # print(np.max(np.abs(CC)), file=sys.stderr)
    # break
    
print()

print('key array:', key_arr)
print('correlation values:', corrs)
    # break

# a1 b1 c1 d1 e1 a2 b2 c2 d2 e2

# first row: 0* 285000 to 1 * 285000
# second row: 1 * 285000 to 2 * 285000

# m[1, 2]
# plt.show()



    
#NOTE: The amount of code you need to write in this section
#is very small. It can reasonably be done in 4 lines of
#code or less...

05deadbeef42006861636b65646b6579
key array: [  5. 222. 173. 190. 239.  66.   0. 104.  97.  99. 107. 101. 100. 107.
 101. 121.]
correlation values: [0.72666294 0.80468848 0.59106522 0.79803863 0.50403607 0.76333627
 0.50855133 0.75113276 0.59413205 0.77256303 0.93029037 0.73536212
 0.62999715 0.79989786 0.82894694 0.79117384]


In [10]:

from Crypto.Cipher import AES
from binascii import hexlify, unhexlify
key = np.uint8(key_arr).tobytes()

cipher = AES.new(key, AES.MODE_ECB)
key_correct = True
# print(myload(ptfile2))
for plaintext, ciphertext_ref in zip(plaintexts, ciphertexts):
    plain = np.uint8(plaintext).tobytes()
    ciphertext = cipher.encrypt(plain)
    cipher_ref = np.uint8(ciphertext_ref).tobytes()
    key_correct &= ciphertext == cipher_ref
    if ciphertext != cipher_ref:
        print('key is incorrect')
        break
    # print(hexlify(cipher_ref))

print(f'key_correct = {key_correct}')


key_correct = True
